## Set-up environment

As usual, we first install HuggingFace Transformers, and Datasets.

In [ ]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import os
from pytorch_lightning.loggers import TensorBoardLogger
%load_ext autoreload
%autoreload 2

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov 17 06:06:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    46W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
!pip install -q datasets

## Prepare data

Here we take a small portion of the IMDB dataset, a binary text classification dataset ("is a movie review positive or negative?").

In [ ]:
from datasets import load_dataset

train_ds, test_ds = load_dataset("imdb", split=['train', 'test[:6250]+test[-6250:]'])
train_ds, val_ds = load_dataset("imdb", split=['train', 'test[6250:12500]+test[-12500:-6250]'])

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(test_ds)
print(val_ds)

Dataset({
    features: ['text', 'label'],
    num_rows: 12500
})
Dataset({
    features: ['text', 'label'],
    num_rows: 12500
})


We create id2label and label2id mappings, which are handy at inference time.

In [ ]:
labels = train_ds.features['label'].names
print(labels)

['neg', 'pos']


In [ ]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
print(id2label)

{0: 'neg', 1: 'pos'}


Next, we prepare the data for the model using the tokenizer. 

In [ ]:
from transformers import PerceiverTokenizer

tokenizer = PerceiverTokenizer.from_pretrained("deepmind/language-perceiver")

train_ds = train_ds.map(lambda examples: tokenizer(examples['text'], padding="max_length", truncation=True),
                        batched=True)
val_ds = val_ds.map(lambda examples: tokenizer(examples['text'], padding="max_length", truncation=True),
                      batched=True)
test_ds = test_ds.map(lambda examples: tokenizer(examples['text'], padding="max_length", truncation=True),
                      batched=True)

Downloading:   0%|          | 0.00/668 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/879 [00:00<?, ?B/s]

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

We set the format to PyTorch tensors, and create familiar PyTorch dataloaders.

In [ ]:
train_ds.set_format(type="torch", columns=['input_ids', 'label'])
test_ds.set_format(type="torch", columns=['input_ids', 'label'])
val_ds.set_format(type="torch", columns=['input_ids', 'label'])

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_ds, batch_size=100, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=25)
val_dataloader = DataLoader(val_ds, batch_size=25)

Here we verify some things (always important to check out your data!).

In [ ]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
  print(k,v.shape)

label torch.Size([100])
input_ids torch.Size([100, 2048])


In [ ]:
tokenizer.decode(batch['input_ids'][3])

"[CLS]I never expect a film adaptation to follow too closely to the novel (especially a beloved one, like Evening) but when I saw that the book's author, Susan Minot, was a screenplay writer and executive producer on the film, I thought that Evening would be a good adaptation.<br /><br />If you enjoyed the book, don't bother with this movie. It is so far afield of the book that the two hardly bear any resemblance to one another.<br /><br />Here, our characters are completely different: the bride is in love with Harris. Harris is the son of the housekeeper. Buddy is a drunk, in love with Ann and/or Harris. I don't think a single character made it from the book to the screen; oh it just gets worst with every passing moment.<br /><br />And, really, didn't we learn from Bridges of Madison County that cutting from the story we are meant to be enthralled in, to scenes of our heroes' grown children having obnoxious and juvenile fights, simply does not work on film? This film is a disaster. Sk

In [ ]:
# batch['label']

In [ ]:
import numpy as np
train_ds['label'].double().mean()
print(train_ds['label'][12499])

tensor(0)


## Define model

Next, we define our model, and put it on the GPU.

In [ ]:
# preprocessor we customized to use the tagkop encoder
from tagkop_encoding_functions import (
    PerceiverImagePreprocessor,
    TagkopPerceiverTextPreprocessor,
)
from transformers import PerceiverForSequenceClassification

import torch

from transformers.models.perceiver.modeling_perceiver import (
    PerceiverConfig,
    PerceiverModel,
    PerceiverClassificationDecoder,
    PerceiverTextPreprocessor,
    PerceiverClassificationDecoder
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


config = PerceiverConfig(
    num_self_attends_per_block = 4,
    d_model = 64
)

print('config', config)

# Vanilla Perceiver Encodings


preprocessor = PerceiverTextPreprocessor(config)

# Our new awesome encodings
# preprocessor = TagkopPerceiverTextPreprocessor(config)


decoder = PerceiverClassificationDecoder(config,
                                          num_channels=config.d_latents,
                                          trainable_position_encoding_kwargs=dict(num_channels=config.d_latents, index_dims=1),
                                          use_query_residual=True,
                                         )

# num_self_attends_per_block, num_self_attention_heads, num_cross_attention_heads to something more reasonable and out_channels project_pos_dim and num_channels to 64
model = PerceiverModel(config, input_preprocessor=preprocessor, decoder=decoder)



model.to(device)

config PerceiverConfig {
  "attention_probs_dropout_prob": 0.1,
  "audio_samples_per_frame": 1920,
  "cross_attention_shape_for_attention": "kv",
  "cross_attention_widening_factor": 1,
  "d_latents": 1280,
  "d_model": 64,
  "hidden_act": "gelu",
  "image_size": 56,
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "perceiver",
  "num_blocks": 1,
  "num_cross_attention_heads": 8,
  "num_frames": 16,
  "num_latents": 256,
  "num_self_attends_per_block": 4,
  "num_self_attention_heads": 8,
  "output_shape": [
    1,
    16,
    224,
    224
  ],
  "qk_channels": null,
  "samples_per_patch": 16,
  "self_attention_widening_factor": 1,
  "train_size": [
    368,
    496
  ],
  "transformers_version": "4.25.0.dev0",
  "use_query_residual": true,
  "v_channels": null,
  "vocab_size": 262
}



PerceiverModel(
  (input_preprocessor): PerceiverTextPreprocessor(
    (embeddings): Embedding(262, 64)
    (position_embeddings): Embedding(2048, 64)
  )
  (embeddings): PerceiverEmbeddings()
  (encoder): PerceiverEncoder(
    (cross_attention): PerceiverLayer(
      (attention): PerceiverAttention(
        (self): PerceiverSelfAttention(
          (layernorm1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (layernorm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (query): Linear(in_features=1280, out_features=64, bias=True)
          (key): Linear(in_features=64, out_features=64, bias=True)
          (value): Linear(in_features=64, out_features=64, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): PerceiverSelfOutput(
          (dense): Linear(in_features=64, out_features=1280, bias=True)
        )
      )
      (layernorm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (mlp): PerceiverMLP(
 

In [ ]:
# you can then do a forward pass as follows:
tokenizer = PerceiverTokenizer()
text = "hello world"
inputs = tokenizer(text, return_tensors="pt").input_ids
print(inputs)
inputs.to(device)
with torch.no_grad():
   outputs = model(inputs=inputs.to(device))
logits = outputs.logits
print('list(logits.shape): ', list(logits.shape))
# to train, one can train the model using standard cross-entropy:
criterion = torch.nn.CrossEntropyLoss()
labels = torch.tensor([1]).to(device)
loss = criterion(logits, labels)

tensor([[  4, 110, 107, 114, 114, 117,  38, 125, 117, 120, 114, 106,   5]])
list(logits.shape):  [1, 2]


In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/saved_model/small_model_Nov_16_epoch_8.pt'))
model.to(device)

PerceiverModel(
  (input_preprocessor): PerceiverTextPreprocessor(
    (embeddings): Embedding(262, 64)
    (position_embeddings): Embedding(2048, 64)
  )
  (embeddings): PerceiverEmbeddings()
  (encoder): PerceiverEncoder(
    (cross_attention): PerceiverLayer(
      (attention): PerceiverAttention(
        (self): PerceiverSelfAttention(
          (layernorm1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (layernorm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (query): Linear(in_features=1280, out_features=64, bias=True)
          (key): Linear(in_features=64, out_features=64, bias=True)
          (value): Linear(in_features=64, out_features=64, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): PerceiverSelfOutput(
          (dense): Linear(in_features=64, out_features=1280, bias=True)
        )
      )
      (layernorm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (mlp): PerceiverMLP(
 

## Train the model

Here we train the model using native PyTorch.

In [ ]:
from transformers import AdamW
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score

from tqdm.notebook import tqdm
from datasets import load_metric

optimizer = AdamW(model.parameters(), lr=1e-4)

model.train()

batch = next(iter(train_dataloader))
for epoch in range(50):  # loop over the dataset multiple times

    epoch_accs = np.array([])
    torch.save(model.state_dict(), '/content/drive/MyDrive/saved_model/rerun_small_model_Nov_16_epoch_' + str(epoch) + '.pt')
    print('saved model')
    print("Epoch:", epoch)
    # for i in range(2):
    for batch in tqdm(train_dataloader):
         batch_accs = np.array([])
         # get the inputs; 
         inputs = batch["input_ids"].to(device)
        #  attention_mask = batch["attention_mask"].to(device)
         labels = batch["label"].to(device)

         # zero the parameter gradients
         optimizer.zero_grad()

         # forward + backward + optimize
         outputs = model(inputs=inputs)
         logits = outputs.logits
         
         # to train, one can train the model using standard cross-entropy:
         criterion = torch.nn.CrossEntropyLoss()

         loss = criterion(logits, labels)
         loss.backward()
         optimizer.step()
         
         # evaluate
         predictions = outputs.logits.argmax(-1).cpu().detach().numpy()
         accuracy = accuracy_score(y_true=batch["label"].numpy(), y_pred=predictions)
         print(f"Loss: {loss.item()}, Accuracy: {accuracy}")
         batch_accs = np.append(batch_accs, accuracy)
    
    print('Batch Accuracy Avg: ', np.mean(batch_accs))
    
    accuracy = load_metric("accuracy")
    
    # Validation Test
    model.eval()
    for batch in tqdm(val_dataloader):
          # get the inputs; 
          inputs = batch["input_ids"].to(device)
          labels = batch["label"].to(device)

          # forward pass
          outputs = model(inputs=inputs)
          logits = outputs.logits 
          predictions = logits.argmax(-1).cpu().detach().numpy()
          references = batch["label"].numpy()
          accuracy.add_batch(predictions=predictions, references=references)

    final_score = accuracy.compute()
    print("Accuracy on validation set:", final_score)
    epoch_accs = np.append(epoch_accs, final_score)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


saved model
Epoch: 0


  0%|          | 0/250 [00:00<?, ?it/s]

Loss: 0.693368136882782, Accuracy: 0.44
Loss: 0.7168745398521423, Accuracy: 0.51
Loss: 0.7018337249755859, Accuracy: 0.44
Loss: 0.7624335289001465, Accuracy: 0.39
Loss: 0.6923819184303284, Accuracy: 0.52
Loss: 0.6946253180503845, Accuracy: 0.5
Loss: 0.7050689458847046, Accuracy: 0.48
Loss: 0.687935471534729, Accuracy: 0.58
Loss: 0.6944180130958557, Accuracy: 0.49
Loss: 0.6934633851051331, Accuracy: 0.49
Loss: 0.69244384765625, Accuracy: 0.49
Loss: 0.6969552636146545, Accuracy: 0.42
Loss: 0.6833515763282776, Accuracy: 0.58
Loss: 0.7242802977561951, Accuracy: 0.49
Loss: 0.7121180891990662, Accuracy: 0.5
Loss: 0.6814194321632385, Accuracy: 0.59
Loss: 0.6940053701400757, Accuracy: 0.47
Loss: 0.6947258710861206, Accuracy: 0.49
Loss: 0.703264057636261, Accuracy: 0.47
Loss: 0.6883248686790466, Accuracy: 0.55
Loss: 0.6932123303413391, Accuracy: 0.5
Loss: 0.6910974979400635, Accuracy: 0.54
Loss: 0.6915602087974548, Accuracy: 0.51
Loss: 0.6999536752700806, Accuracy: 0.41
Loss: 0.6959879398345947

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate


...

Accuracy on validation set: {'accuracy': 0.60144}
saved model
Epoch: 49


  0%|          | 0/250 [00:00<?, ?it/s]

Loss: 0.12841780483722687, Accuracy: 0.93
Loss: 0.08401262015104294, Accuracy: 0.97
Loss: 0.1365337073802948, Accuracy: 0.93
Loss: 0.08215785771608353, Accuracy: 0.97
Loss: 0.06760230660438538, Accuracy: 0.96
Loss: 0.14563098549842834, Accuracy: 0.94
Loss: 0.09546879678964615, Accuracy: 0.95
Loss: 0.0674472376704216, Accuracy: 0.98
Loss: 0.1319706290960312, Accuracy: 0.97
Loss: 0.10262922942638397, Accuracy: 0.95
Loss: 0.08252410590648651, Accuracy: 0.99
Loss: 0.08442489802837372, Accuracy: 0.96
Loss: 0.09942060708999634, Accuracy: 0.94
Loss: 0.07538425177335739, Accuracy: 0.96
Loss: 0.043820735067129135, Accuracy: 0.99
Loss: 0.11426311731338501, Accuracy: 0.97
Loss: 0.07761441171169281, Accuracy: 0.96
Loss: 0.04873674735426903, Accuracy: 0.99
Loss: 0.10058242827653885, Accuracy: 0.94
Loss: 0.08797258138656616, Accuracy: 0.97
Loss: 0.02841714769601822, Accuracy: 0.98
Loss: 0.060032982379198074, Accuracy: 0.98
Loss: 0.06743095070123672, Accuracy: 0.97
Loss: 0.10952449589967728, Accuracy

  0%|          | 0/500 [00:00<?, ?it/s]

Accuracy on validation set: {'accuracy': 0.59728}


## Evaluate the model

Finally, we evaluate the model on the test set. We use the Datasets library to compute the accuracy.

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/saved_model/rerun_small_model_Nov_16_epoch_last.pt')

In [ ]:

# import torch
# checkpoint = torch.load('/content/drive/MyDrive/saved_model/small_network_model.pt')
# model.load_state_dict(checkpoint)
# model.eval()

In [ ]:
from tqdm.notebook import tqdm
from datasets import load_metric

accuracy = load_metric("accuracy")

model.eval()
for batch in tqdm(test_dataloader):
      # get the inputs; 
      inputs = batch["input_ids"].to(device)
      # attention_mask = batch["attention_mask"].to(device)
      labels = batch["label"].to(device)

      # forward pass
      outputs = model(inputs=inputs)
      logits = outputs.logits 
      predictions = logits.argmax(-1).cpu().detach().numpy()
      references = batch["label"].numpy()
      accuracy.add_batch(predictions=predictions, references=references)

final_score = accuracy.compute()
print("Accuracy on test set:", final_score)

  0%|          | 0/500 [00:00<?, ?it/s]

Accuracy on test set: {'accuracy': 0.5928}


## Inference

In [ ]:
text = "I hated this movie, it's really bad."

input_ids = tokenizer(text, return_tensors="pt").input_ids

# forward pass
outputs = model(inputs=input_ids.to(device))
logits = outputs.logits 
predicted_class_idx = logits.argmax(-1).item()

print("Predicted:", model.config.id2label[predicted_class_idx])